In [39]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


import seaborn as sns
import sklearn
import plotly.graph_objects as go
import plotly.express as px
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline



#Importing all the req libraries
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import classification_report 
from sklearn.metrics import precision_score, accuracy_score , recall_score , roc_auc_score


In [40]:
training_data= pd.read_csv('train.csv')
testing_data= pd.read_csv('test.csv')


In [41]:
training_data

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


In [42]:
training_data.describe(include='O')

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,VIP,Name
count,8693,8492,8476,8494,8511,8490,8493
unique,8693,3,2,6560,3,2,8473
top,0001_01,Earth,False,G/734/S,TRAPPIST-1e,False,Gollux Reedall
freq,1,4602,5439,8,5915,8291,2


In [43]:
training_data.isnull().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
               ... 
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
Length: 14, dtype: int64

In [44]:
label=training_data.iloc[:,13]
  #Droping the label from training data set
training_data.drop(['Transported'], axis=1, inplace=True)
label


0       False
1        True
2       False
3       False
4        True
        ...  
8688    False
8689    False
8690     True
8691    False
8692     True
Name: Transported, Length: 8693, dtype: bool

In [45]:
def clean(data):
   
    
    split=data['Cabin'].str.split('/',expand=True)
    split= split.rename(columns={0: 'Deck', 1: 'Number',2: 'Side'})
    split.drop(['Number'], axis=1, inplace=True)
    
    

    
    lol =data["PassengerId"].str[0:4]
    data['VIP'] = data['VIP'].astype(float)
    data['CryoSleep'] = data['CryoSleep'].astype(float)
    data['CryoSleep']=data['CryoSleep'].replace(('False','True'),(0,1))
    data['VIP']=data['VIP'].replace(('False','True'),(0,1))
    
    
    data.drop(['Cabin'], axis=1, inplace=True)
    data.drop(['Name'], axis=1, inplace=True)
    data.drop(['PassengerId'], axis=1, inplace=True)
    
    
   
    
    # takes mean and mode of numerical as well as categorical variables to fill missing values
    data=data.apply(lambda x:x.fillna(x.mean()) if x.dtype=='float' else x.fillna(x.value_counts().index[0]))   
    split=split.apply(lambda x:x.fillna(x.mean()) if x.dtype=='float' else x.fillna(x.value_counts().index[0]))
    
    clean_train=pd.concat([data,split], axis=1)
    clean_data=pd.get_dummies(clean_train,drop_first=True)
    #clean_data=pd.concat([clean_data,lol], axis=1)
    return (clean_data)

In [46]:
clean_data=clean(training_data)

In [47]:
clean_data

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Europa,HomePlanet_Mars,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Deck_T,Side_S
0,0.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,1,1,0,0,0,0,0,0,0
1,0.0,24.0,0.0,109.0,9.0,25.0,549.0,44.0,0,0,0,1,0,0,0,0,1,0,0,1
2,0.0,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0,1,0,0,1,0,0,0,0,0,0,0,1
3,0.0,33.0,0.0,0.0,1283.0,371.0,3329.0,193.0,1,0,0,1,0,0,0,0,0,0,0,1
4,0.0,16.0,0.0,303.0,70.0,151.0,565.0,2.0,0,0,0,1,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,0.0,41.0,1.0,0.0,6819.0,0.0,1643.0,74.0,1,0,0,0,0,0,0,0,0,0,0,0
8689,1.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,0,0,0,0,0,1,0,1
8690,0.0,26.0,0.0,0.0,0.0,1872.0,1.0,0.0,0,0,0,1,0,0,0,0,0,1,0,1
8691,0.0,32.0,0.0,0.0,1049.0,0.0,353.0,3235.0,1,0,0,0,0,0,0,1,0,0,0,1


In [ ]:
corr = clean_data.corr()
plt.figure(figsize=(15, 15))
sns.set(font_scale=0.8)
sns.heatmap(corr, square=True, cmap="PRGn", linewidths=2, annot=True, cbar_kws={'shrink': 0.8})
plt.title("Correlation Matrix", fontsize=16, pad=30)
plt.show()

In [ ]:
#Visualising Data to see patterns/correlations
pd.crosstab(index=training_data['HomePlanet'], columns='label', dropna=True)

In [48]:
X_train, X_val, y_train, y_val = train_test_split(clean_data, label, test_size = 0.30, random_state = 0)

In [49]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
#model = make_pipeline(StandardScaler(),SVC(gamma="float"))

In [50]:
#create parameter grid for model tuning (SVC)
kernel = ['linear', 'poly', 'rbf','sigmoid']
gamma = ['scale', 'auto']

random_grid ={
    'kernel' : kernel,
    'gamma' : gamma
}

In [13]:
#apply RandomizedSearchCV to get best parameters for LR with CV
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn import svm
model_svc = SVC()

SVC_randomcv = RandomizedSearchCV(estimator = model_svc, 
                               param_distributions = random_grid,
                               n_iter = 3, cv = 3, 
                               random_state=10)

In [14]:
SVC_randomcv.fit(X_train, y_train)

RandomizedSearchCV(cv=3, estimator=SVC(), n_iter=3,
                   param_distributions={'gamma': ['scale', 'auto'],
                                        'kernel': ['linear', 'poly', 'rbf',
                                                   'sigmoid']},
                   random_state=10)

In [15]:
SVC_randomcv.best_params_

{'kernel': 'rbf', 'gamma': 'scale'}

In [66]:
model_svc = SVC(kernel='rbf',gamma = 'scale')
knn=KNeighborsClassifier(n_neighbors=20)

knn.fit(X_train, y_train)
y_pred = knn.predict(X_val)
print("-------SVC--------")
print("Accuracy : ", accuracy_score(y_pred,y_val))
print("Precision : ", precision_score(y_pred,y_val))
print("Recall : ", recall_score(y_pred,y_val))
print("AUC score: ",roc_auc_score(y_pred,y_val))

-------SVC--------
Accuracy :  0.7864263803680982
Precision :  0.8306513409961686
Recall :  0.7633802816901408
AUC score:  0.7886766728316024


In [19]:
model.fit(X_train,y_train) 

-------SVC--------
Accuracy :  0.7772239263803681
Precision :  0.9080459770114943
Recall :  0.7199270959902795
AUC score:  0.7975934856250774

SyntaxError: invalid syntax (128518870.py, line 3)

In [59]:
from sklearn.neighbors import KNeighborsClassifier
misscalssified_sample=[]
for i in range (5,30):
    knn=KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train, y_train)
    pred_i=knn.predict(X_val)
    misscalssified_sample.append((y_val!=pred_i).sum())

In [60]:
predict=model.predict(X_val)

In [62]:
misscalssified_sample

[594,
 580,
 563,
 564,
 567,
 567,
 578,
 554,
 564,
 568,
 569,
 572,
 559,
 559,
 550,
 557,
 552,
 562,
 565,
 555,
 566,
 560,
 567,
 569,
 565]

In [61]:
print(classification_report(y_val,predict))
print('AUC = %.2f' % roc_auc_score(y_val, predict))
print("accuracy: ", accuracy_score(y_val, predict))
print("precison: ", precision_score(y_val, predict))
print("RECALL: ", recall_score(y_val, predict))


              precision    recall  f1-score   support

       False       0.82      0.80      0.81      1303
        True       0.80      0.82      0.81      1305

    accuracy                           0.81      2608
   macro avg       0.81      0.81      0.81      2608
weighted avg       0.81      0.81      0.81      2608

AUC = 0.81
accuracy:  0.8098159509202454
precison:  0.8025430067314884
RECALL:  0.8222222222222222


In [52]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
model = make_pipeline(StandardScaler(),model_svc)


In [53]:
model.fit(clean_data,label)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('kneighborsclassifier', KNeighborsClassifier(n_neighbors=19))])

In [54]:
clean_test_data=clean(testing_data)

In [55]:
result=model.predict(clean_test_data)

In [56]:
result

array([ True, False,  True, ...,  True,  True,  True])

In [57]:
sample= pd.read_csv('sample_submission.csv')

sample1=sample.iloc[:,0]

In [58]:
df = pd.DataFrame(result, columns = ['Transported'])
final_result=pd.concat([sample1,df], axis=1)

final_result.to_csv('model1.csv', index = None)